# ANAC


In [16]:
import pandas as pd
import numpy as np

#### set seed

In [12]:
seed = 2024
np.random.seed(seed)

#### read in data

In [14]:
diet_csv = pd.read_csv("diet.csv") 
recipes_csv = pd.read_csv("recipes.csv")
requests_csv = pd.read_csv("requests.csv")

In [17]:
#it seems that a column in 'review' contains a combination of numeric and non-numeric values.
reviews_csv = pd.read_csv("reviews.csv")
#inspect the types of each column
reviews_csv.info()
#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AuthorId   140195 non-null  object 
 1   RecipeId   140195 non-null  int64  
 2   Rating     77108 non-null   float64
 3   Like       97381 non-null   object 
 4   TestSetId  42814 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.3+ MB


/var/folders/69/1cfwzk1s0fv572_7rpfvjk7m0000gn/T/ipykernel_17905/3669628886.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_csv = pd.read_csv("reviews.csv")


### Data Cleaning
#### diet_csv

In [8]:
diet_csv["Diet"] = diet_csv["Diet"].astype("category")

#### recipes_csv

In [9]:
# TODO CookTime, PrepTime - is it in Minutes/Seconds? RecipeIngredientParts & RecipeIngredientQuantities
# TODO What to do with Servings and Yield?
recipes_csv["RecipeCategory"] = recipes_csv["RecipeCategory"].astype("category")

##### requests_csv

In [10]:
# TODO Time - Is it in Minutes/Seconds?
requests_csv["HighCalories"] = requests_csv["HighCalories"].astype("boolean")
requests_csv["LowFat"] = requests_csv["LowFat"].astype("boolean")
requests_csv["HighFiber"] = requests_csv["HighFiber"].astype("boolean")

requests_csv["HighProtein"] = requests_csv["HighProtein"].map({
    "Indifferent": 0,
    "0": 0,
    "1": 1,
    "Yes": 1
})
requests_csv["LowSugar"] = requests_csv["LowSugar"].map({
    "Indifferent": 0,
    "0": 0,
    "1": 1
})
requests_csv["HighProtein"] = requests_csv["HighProtein"].astype("boolean")
requests_csv["LowSugar"] = requests_csv["LowSugar"].astype("boolean")

#### reviews_csv

In [ ]:
reviews_csv.info()
# TODO wtf rating only has values 2/NA -> delete? 
# TODO need to fix NA values for Like -> delete rows with NA?
# TODO why are there missing values for TestSetId? A: because for training phase we don't need the TestSetId, for prediction and testing phase we have the TestSetId
#print(reviews_csv["Rating"].values.unique())

# TODO Join the tables